In [ ]:
import torch

#2.1.1. 入门
#使用 arange 创建一个行向量 x (0-11)
x = torch.arange(12)
#可以通过张量的 shape 属性来访问张量的 形状
x.shape
#只想知道张量中元素的总数，即形状的所有元素乘积，我们可以检查它的大小（size）
x.numel()
#张量 x 从形状为 (12, ) 的行向量转换为形状 (3, 4) 的矩阵
x.reshape(3, 4)
#如果我们的目标形状是 (高度, 宽度) ，那么在知道宽度后，高度应当会隐式得出，我们不必自己做除法
#x.reshape(-1, 4) 或 x.reshape(3, -1)

#创建一个形状为 (2, 3, 4) 的张量，其中所有元素都设置为0
torch.zeros((2, 3, 4))
#创建一个张量，其中所有元素都设置为1
torch.ones((2, 3, 4))
#创建一个形状为 (3, 4) 的张量。其中的每个元素都从均值为0、标准差为1的标准高斯（正态）分布中随机采样。
torch.randn(3, 4)
#通过提供包含数值的 Python 列表（或嵌套列表）来为所需张量中的每个元素赋予确定值。
torch.tensor([[2, 1, 4, 3], [1, 2, 3, 4], [4, 3, 2, 1]])

In [ ]:
#2.1.2. 运算
#使用逗号来表示一个具有5个元素的元组，其中每个元素都是按元素操作的结果。
x = torch.tensor([1.0, 2, 4, 8])
y = torch.tensor([2, 2, 2, 2])
x + y, x - y, x * y, x / y, x**y  # **运算符是求幂运算
#可以按按元素方式应用更多的计算，包括像求幂这样的一元运算符。
torch.exp(x)
#分别演示了当我们沿行（轴-0，形状的第一个元素）和按列（轴-1，形状的第二个元素）连结两个矩阵时会发生什么情况
X = torch.arange(12, dtype=torch.float32).reshape((3, 4))
Y = torch.tensor([[2.0, 1, 4, 3], [1, 2, 3, 4], [4, 3, 2, 1]])
torch.cat((X, Y), dim=0), torch.cat((X, Y), dim=1)
#以 X == Y 为例子。 对于每个位置，如果 X 和 Y 在该位置相等，则新张量中相应项的值为1，这意味着逻辑语句 X == Y 在该位置处为真，否则该位置为 0。
X == Y
#对张量中的所有元素进行求和会产生一个只有一个元素的张量。
X.sum()

In [ ]:
#2.1.3. 广播机制
#在大多数情况下，我们将沿着数组中长度为1的轴进行广播
a = torch.arange(3).reshape((3, 1))
b = torch.arange(2).reshape((1, 2))
a, b
#由于 a 和 b 分别是  3×1  和  1×2  矩阵，如果我们让它们相加，它们的形状不匹配。我们将两个矩阵广播为一个更大的  3×2  矩阵，如下所示：矩阵 a将复制列，矩阵 b将复制行，然后再按元素相加。
a + b

In [ ]:
#2.1.4. 索引和切片
#以用 [-1] 选择最后一个元素，可以用 [1:3] 选择第二个和第三个元素，如下所示：
X[-1], X[1:3]
#除读取外，我们还可以通过指定索引来将元素写入矩阵。
X[1, 2] = 9
X
#如果我们想为多个元素赋值相同的值，我们只需要索引所有元素，然后为它们赋值。 例如，[0:2, :] 访问第1行和第2行，其中 “:” 代表沿轴 1（列）的所有元素。
X[0:2, :] = 12
X

In [ ]:
#2.1.5. 节省内存
X = torch.arange(3).reshape((3, 1))
Y = torch.arange(2).reshape((1, 2))
#运行 Y = Y + X 后，我们会发现 id(Y) 指向另一个位置。这是因为 Python 首先计算 Y + X，为结果分配新的内存，然后使 Y 指向内存中的这个新位置。
before = id(Y)
Y = Y + X
id(Y) == before
#幸运的是，执行原地操作非常简单。我们可以使用切片表示法将操作的结果分配给先前分配的数组，例如 Y[:] = <expression>。为了说明这一点，我们首先创建一个新的矩阵 Z，其形状与另一个 Y 相同，使用 zeros_like 来分配一个全 0 的块。
Z = torch.zeros_like(Y)
print('id(Z):', id(Z))
Z[:] = X + Y
print('id(Z):', id(Z))
#如果在后续计算中没有重复使用 X，我们也可以使用 X[:] = X + Y 或 X += Y 来减少操作的内存开销。
before = id(X)
X += Y
id(X) == before

In [ ]:
#先用zeros_like(Y)產生Z, 用另上面的兩個方式去减少操作的内存开销, 用id()打印位置
X = torch.arange(3)
Y = torch.arange(3)

Z = torch.zeros_like(Y)
print(id(Z))

Z[:]  = X + Y
print(id(Z))

Z += X
Z += Y
print(id(Z))

In [ ]:
#2.1.6. 转换为其他 Python 对象
#转换为 NumPy 张量很容易，反之也很容易。转换后的结果不共享内存。 这个小的不便实际上是非常重要的：当你在 CPU 或 GPU 上执行操作的时候，此时Python的NumPy包也希望使用相同的内存块执行其他操作时，你不希望停止计算。
A = X.numpy()
B = torch.tensor(A)
type(A), type(B)
#要将大小为1的张量转换为 Python 标量，我们可以调用 item 函数或 Python 的内置函数。
a = torch.tensor([3.5])
a, a.item(), float(a), int(a)